In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import itertools
import csv,codecs,nltk,re
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import itertools
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
#from sklearn import cross_validation#    from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, NuSVC , LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [2]:
from sklearn.model_selection import KFold

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier


In [4]:
#data2 = pd.read_csv('E:\cluster data\One_File_nonnormalizenew2.txt', sep="*")
data = pd.read_csv('E:\\new data2019\\One_File_44000 clean.txt', sep="*")

data.columns = ["text", "class1"]
data.dropna(inplace=True)
#data2.columns = ["text", "class"]

In [5]:
len(data)

44331

In [6]:
data.class1.value_counts()

tv          4056
ads         4040
politic     4037
tech        4036
food        4032
health      4028
porno       4026
eco         4024
religion    4020
weather     4020
sport       4012
Name: class1, dtype: int64

In [7]:
data=data.groupby('class1').apply(lambda x: x.iloc[:4000])

In [8]:
data.class1.value_counts()

religion    4000
sport       4000
tech        4000
food        4000
tv          4000
politic     4000
eco         4000
ads         4000
health      4000
porno       4000
weather     4000
Name: class1, dtype: int64

In [9]:
def normalize(text):
    text = re.sub("[إﺁأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("_", " ", text)
    text = re.sub("#", " ", text)
    
    text = re.sub("ؤ", "و", text)
    #text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    return(text)
def stopwordremoval(text):
    stop=stopwords.words("arabic - Copy")
    needed_words=[]
    words=word_tokenize(text)
    for w in words:
         if len(w)>=2 and w not in stop:
                needed_words.append(w)
    filterd_sent= " ".join(needed_words)
    return filterd_sent
def removenonarabic(text):
    n=re.sub(r'[a-zA-Z?]', '', text).strip()
    n=re.sub('\W+',' ', n )
    n=re.sub('_','', n )
    n = ''.join([i for i in n if not i.isdigit()])
    a=re.compile(u"([^\u0600-\u06ff\ufb50-\ufdff\ufe70-\ufeff\u0750-\u077f ])", re.UNICODE)
    n=re.sub(a,'', n)
    return n
def stemming(text):
    st = ISRIStemmer()
    stemmed_words=[]
    words=word_tokenize(text)
    for w in words:
        stemmed_words.append(st.stem(w))
    stemmed_sent=" ".join(stemmed_words)
    return stemmed_sent
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
def preparedatasets(data):
    sentences=[]
    for index,r in data.iterrows():
        text=normalize(r['text'])
        text=stopwordremoval(text)
        text=removenonarabic(text)
        #text=stemming(text)
        sentences.append([text,r['class1']])
    df_sentence=DataFrame(sentences, columns=["text", "class1"])
    return df_sentence        

In [10]:
data=preparedatasets(data)


In [11]:
X=data['text']
y=data['class1']

In [12]:
 X_train, X_test, y_train, y_test = \
    train_test_split(data['text'], data['class1'],test_size=0.25, random_state = 42)

In [13]:
X_train.dropna(inplace=True)
y_train.dropna(inplace=True)

len(X_train)

33000

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score

In [15]:
cl=[MultinomialNB, BernoulliNB,LogisticRegression,SGDClassifier,SVC, NuSVC , LinearSVC,DecisionTreeClassifier]

In [16]:
def get_count(data):
    cc = pd.DataFrame()
    cc['countt']=[len(t.split())for t in data.text] 
    

    return cc.sum().countt    

In [17]:
print(get_count(data))

748827


In [13]:
#dd=data[data['class1']=='politic']
#dd.to_csv("e:\\politic.txt", sep='\t', encoding='utf-8')

# Feature Extraction

In [1]:
def learning(clf,X,Y):
        X_train, X_text, Y_train, Y_test=\
        cross_validation.train_test_split(X,Y,test_size=0.1, random_state=43)
        #print("Y train ",X_train)
        classifier=clf()
        classifier.fit(X_train, Y_train)
        predict=cross_validation.cross_val_predict(classifier,X_text,Y_test, cv=10)#cv=10
        scores=cross_validation.cross_val_score(classifier,X_text,Y_test, cv=10)
       # print(Y_test,predict)
        print("Accuracy of %s: %0.2f (+/- %0.2f)"%(classifier, scores.mean(),scores.std()*2))#standard deviation
        print(classification_report(Y_test,predict))

In [ ]:
def main(clf):
   # data = pd.read_csv('e:\One_File_nonnormalizenew2norm.txt', sep="*")
  #  data.columns = ["text", "class"]
  #  data.dropna(inplace=True)
  #  new_data=preparedatasets(data)
    data, target=data['text'], new_data['class1']
    #print("class",target)
    tfidf_data=featureextraction(data)
   # print(tfidf_data)
    learning(clf,tfidf_data,target)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import *
result="Classifier"+'*'+"dataset_size * Accuracy * f1_score * Recall * precision"+"\n"
def featureextraction2(data , clf):
    data.dropna(inplace=True)
    global result
    X_train, X_test, y_train, y_test = \
    train_test_split(data['text'], data['class1'],test_size=0.20, random_state = 0)
    vectorizer = TfidfVectorizer( analyzer='word',smooth_idf=True, ngram_range=(1,1))
    vectorizer.fit(X_train)
    X_train_vectorized = vectorizer.transform(X_train)
    #clf = LogisticRegression()
    clf.fit(X_train_vectorized, y_train)
    predictions = clf.predict(vectorizer.transform(X_test))
    result +=clf.__class__.__name__+'*'+str(get_count(data))+'*'+str(accuracy_score(y_test, predictions))+'*'+str(f1_score(y_test,predictions,average='weighted'))+'*'+str(recall_score(y_test, predictions,average='weighted'))+'*'+str(precision_score(y_test, predictions,average='weighted'))+'\n'
    print(clf.__class__.__name__)
    print("accuracy",accuracy_score(y_test, predictions))
    print("f1_score",f1_score(y_test, predictions,average='weighted'))
    print("recall",recall_score(y_test, predictions,average='weighted'))
    print("precision",precision_score(y_test, predictions,average='weighted'))    
    print ('\n clasification report:\n', classification_report(y_test, predictions))
    print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

    
    
    
    
    

In [19]:
from sklearn.ensemble import RandomForestClassifier

# Aplly 9 classifier in the dataset

In [20]:
cl=[MultinomialNB, BernoulliNB,LogisticRegression,SGDClassifier,SVC, NuSVC , LinearSVC,DecisionTreeClassifier,RandomForestClassifier]

In [23]:
for classifier in cl:
    featureextraction2(data,classifier())

MultinomialNB
accuracy 0.9682954545454545
f1_score 0.9681784826972638
recall 0.9682954545454545
precision 0.9682389370159301

 clasification report:
               precision    recall  f1-score   support

         ads       0.96      0.99      0.98       795
         eco       0.94      0.91      0.93       821
        food       1.00      0.99      0.99       820
      health       0.96      0.99      0.98       794
     politic       0.94      0.92      0.93       790
       porno       0.99      0.99      0.99       809
    religion       0.99      0.99      0.99       780
       sport       0.98      0.98      0.98       831
        tech       0.97      0.95      0.96       816
          tv       0.95      0.97      0.96       754
     weather       0.97      0.97      0.97       790

   micro avg       0.97      0.97      0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800

 confussion matrix:
 [[789   0   1   

C:\Users\AhOmar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\AhOmar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression
accuracy 0.9623863636363637
f1_score 0.962788075382401
recall 0.9623863636363637
precision 0.9639803609228108

 clasification report:
               precision    recall  f1-score   support

         ads       0.98      0.98      0.98       795
         eco       0.93      0.90      0.91       821
        food       0.99      0.98      0.99       820
      health       0.96      0.98      0.97       794
     politic       0.84      0.95      0.89       790
       porno       1.00      0.99      0.99       809
    religion       0.98      0.98      0.98       780
       sport       0.99      0.97      0.98       831
        tech       0.96      0.94      0.95       816
          tv       0.99      0.96      0.97       754
     weather       0.99      0.96      0.98       790

   micro avg       0.96      0.96      0.96      8800
   macro avg       0.96      0.96      0.96      8800
weighted avg       0.96      0.96      0.96      8800

 confussion matrix:
 [[778   2   

C:\Users\AhOmar\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier
accuracy 0.9681818181818181
f1_score 0.9682001613368147
recall 0.9681818181818181
precision 0.9684945661652521

 clasification report:
               precision    recall  f1-score   support

         ads       0.98      0.98      0.98       795
         eco       0.94      0.90      0.92       821
        food       0.99      0.99      0.99       820
      health       0.96      0.98      0.97       794
     politic       0.89      0.94      0.91       790
       porno       1.00      1.00      1.00       809
    religion       0.98      0.99      0.98       780
       sport       0.98      0.98      0.98       831
        tech       0.96      0.95      0.95       816
          tv       0.99      0.96      0.98       754
     weather       0.98      0.97      0.98       790

   micro avg       0.97      0.97      0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800

 confussion matrix:
 [[782   1   2   

...

SVC
accuracy 0.08568181818181818
f1_score 0.013523988048794414
recall 0.08568181818181818
precision 0.007341373966942148


C:\Users\AhOmar\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 clasification report:
               precision    recall  f1-score   support

         ads       0.00      0.00      0.00       795
         eco       0.00      0.00      0.00       821
        food       0.00      0.00      0.00       820
      health       0.00      0.00      0.00       794
     politic       0.00      0.00      0.00       790
       porno       0.00      0.00      0.00       809
    religion       0.00      0.00      0.00       780
       sport       0.00      0.00      0.00       831
        tech       0.00      0.00      0.00       816
          tv       0.09      1.00      0.16       754
     weather       0.00      0.00      0.00       790

   micro avg       0.09      0.09      0.09      8800
   macro avg       0.01      0.09      0.01      8800
weighted avg       0.01      0.09      0.01      8800

 confussion matrix:
 [[  0   0   0   0   0   0   0   0   0 795   0]
 [  0   0   0   0   0   0   0   0   0 821   0]
 [  0   0   0   0   0   0   0   0   0 820   0]


C:\Users\AhOmar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
accuracy 0.9069318181818182
f1_score 0.9072619536786113
recall 0.9069318181818182
precision 0.9093511145870439

 clasification report:
               precision    recall  f1-score   support

         ads       0.95      0.95      0.95       795
         eco       0.85      0.84      0.84       821
        food       0.95      0.95      0.95       820
      health       0.91      0.91      0.91       794
     politic       0.83      0.80      0.81       790
       porno       0.98      0.97      0.97       809
    religion       0.96      0.96      0.96       780
       sport       0.96      0.91      0.93       831
        tech       0.91      0.85      0.88       816
          tv       0.93      0.89      0.91       754
     weather       0.78      0.94      0.85       790

   micro avg       0.91      0.91      0.91      8800
   macro avg       0.91      0.91      0.91      8800
weighted avg       0.91      0.91      0.91      8800

 confussion matrix:
 [[754  

In [24]:
from pandas.compat import StringIO

nn = pd.read_csv(StringIO(result), sep='*')
nn

,Classifier,dataset_size,Accuracy,f1_score,Recall,precision
0,MultinomialNB,748827,0.968295,0.968178,0.968295,0.968239
1,BernoulliNB,748827,0.939318,0.939861,0.939318,0.945708
2,LogisticRegression,748827,0.962386,0.962788,0.962386,0.963980
3,SGDClassifier,748827,0.968182,0.968200,0.968182,0.968495
4,SVC,748827,0.085682,0.013524,0.085682,0.007341
5,NuSVC,748827,0.802386,0.819253,0.802386,0.880719
6,LinearSVC,748827,0.970568,0.970687,0.970568,0.971042
7,DecisionTreeClassifier,748827,0.868864,0.869542,0.868864,0.873145
8,RandomForestClassifier,748827,0.906932,0.907262,0.906932,0.909351


# N-Gram

In [25]:
def featureextraction1Gram(data , clf):
    X_train, X_test, y_train, y_test = \
    train_test_split(data['text'], data['class1'],test_size=0.20, random_state = 0)
    vectorizer = TfidfVectorizer( analyzer='word',smooth_idf=True, ngram_range=(1,2))#, norm='l2'.strip_accents='unicode',
    vectorizer.fit(X_train)
    #vectorizer.transform(X_test)
    X_train_vectorized = vectorizer.transform(X_train)
    clf.fit(X_train_vectorized, y_train)
    predictions = clf.predict(vectorizer.transform(X_test))
    global result
    result +="12gram"+'*'+str(len(data))+'*'+str(accuracy_score(y_test, predictions))+'*'+str(f1_score(y_test,predictions,average='weighted'))+'*'+str(recall_score(y_test, predictions,average='weighted'))+'*'+str(precision_score(y_test, predictions,average='weighted'))+'\n'
    print(clf.__class__.__name__)
    print("accuracy",accuracy_score(y_test, predictions))
    print("f1_score",f1_score(y_test, predictions,average='weighted'))
    print("recall",recall_score(y_test, predictions,average='weighted'))
    print("precision",precision_score(y_test, predictions,average='weighted'))    
    print ('\n clasification report:\n', classification_report(y_test, predictions))
    print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

In [26]:
def featureextraction2Gram(data , clf):
    X_train, X_test, y_train, y_test = \
    train_test_split(data['text'], data['class1'],test_size=0.20, random_state = 0)
    vectorizer = TfidfVectorizer( analyzer='word',smooth_idf=True, ngram_range=(2,2))#, norm='l2'.strip_accents='unicode',
    vectorizer.fit(X_train)
    #vectorizer.transform(X_test)
    X_train_vectorized = vectorizer.transform(X_train)
    clf.fit(X_train_vectorized, y_train)
    predictions = clf.predict(vectorizer.transform(X_test))
    global result 
    result +="bigram"+'*'+str(len(data))+'*'+str(accuracy_score(y_test, predictions))+'*'+str(f1_score(y_test,predictions,average='weighted'))+'*'+str(recall_score(y_test, predictions,average='weighted'))+'*'+str(precision_score(y_test, predictions,average='weighted'))+'\n'
    print(clf.__class__.__name__)
    print("accuracy",accuracy_score(y_test, predictions))
    print("f1_score",f1_score(y_test, predictions,average='weighted'))
    print("recall",recall_score(y_test, predictions,average='weighted'))
    print("precision",precision_score(y_test, predictions,average='weighted'))    
    print ('\n clasification report:\n', classification_report(y_test, predictions))
    print (' confussion matrix:\n',confusion_matrix(y_test, predictions))

In [27]:
featureextraction1Gram(data,MultinomialNB())

MultinomialNB
accuracy 0.9672727272727273
f1_score 0.9671016161129923
recall 0.9672727272727273
precision 0.9671681061977739

 clasification report:
               precision    recall  f1-score   support

         ads       0.96      0.99      0.98       795
         eco       0.94      0.90      0.92       821
        food       1.00      0.99      0.99       820
      health       0.96      0.99      0.97       794
     politic       0.93      0.91      0.92       790
       porno       0.99      0.99      0.99       809
    religion       0.99      0.99      0.99       780
       sport       0.98      0.98      0.98       831
        tech       0.97      0.94      0.96       816
          tv       0.95      0.98      0.96       754
     weather       0.97      0.98      0.97       790

   micro avg       0.97      0.97      0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800

 confussion matrix:
 [[790   0   0   

In [28]:
featureextraction2Gram(data,MultinomialNB())

MultinomialNB
accuracy 0.8922727272727272
f1_score 0.8976661068575132
recall 0.8922727272727272
precision 0.9172489100055146

 clasification report:
               precision    recall  f1-score   support

         ads       0.94      0.96      0.95       795
         eco       0.90      0.79      0.84       821
        food       0.98      0.87      0.92       820
      health       0.94      0.90      0.92       794
     politic       0.90      0.71      0.80       790
       porno       0.99      0.96      0.98       809
    religion       0.99      0.94      0.96       780
       sport       0.97      0.92      0.94       831
        tech       0.95      0.84      0.89       816
          tv       0.55      0.97      0.70       754
     weather       0.95      0.96      0.95       790

   micro avg       0.89      0.89      0.89      8800
   macro avg       0.92      0.89      0.90      8800
weighted avg       0.92      0.89      0.90      8800

 confussion matrix:
 [[767   4   0   

In [29]:
from pandas.compat import StringIO

nn = pd.read_csv(StringIO(result), sep='*')

In [30]:
nn

,Classifier,dataset_size,Accuracy,f1_score,Recall,precision
0,MultinomialNB,748827,0.968295,0.968178,0.968295,0.968239
1,BernoulliNB,748827,0.939318,0.939861,0.939318,0.945708
2,LogisticRegression,748827,0.962386,0.962788,0.962386,0.963980
3,SGDClassifier,748827,0.968182,0.968200,0.968182,0.968495
4,SVC,748827,0.085682,0.013524,0.085682,0.007341
5,NuSVC,748827,0.802386,0.819253,0.802386,0.880719
6,LinearSVC,748827,0.970568,0.970687,0.970568,0.971042
7,DecisionTreeClassifier,748827,0.868864,0.869542,0.868864,0.873145
8,RandomForestClassifier,748827,0.906932,0.907262,0.906932,0.909351
9,12gram,44000,0.967273,0.967102,0.967273,0.967168
